## 대리점 데이터로 데이터를 가공

In [91]:
import warnings
warnings.filterwarnings('ignore')

### 1: 데이터 읽기

In [92]:
import pandas as pd

# uriage.csv: 매출이력(기간: 2019/1월~2019/7월)
uriage_data=pd.read_csv(r'C:\유비온_project1\03. 데이터분석\03. 시각화\04.Marketing\02.대리점데이터분석\Data-in\uriage.csv')
uriage_data.head()



,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [93]:
#kokyaku_daicho.xlsx: 대리점에서 관리하는 고객 정보
kokyaku_data=pd.read_excel(r'C:\유비온_project1\03. 데이터분석\03. 시각화\04.Marketing\02.대리점데이터분석\Data-in\kokyaku_daicho.xlsx')
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


### 2: 데이터 오류 확인

In [94]:
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

### 3: 데이터 오류가 있는 상태로 집계

In [95]:
#'매출 이력'에서 상품별로 월 매출 합계를 집계 
# index : purchase_month(년월만추출), columns: item_name, size

#결과: 원래 상품은 26개, 99개 출력으로 나옴 

# 1. uriage_data["purchase_date"] -> date으로 변환
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])

# 2. starftime의 %Y%m 형식으로 변환
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")

# 3. 집계
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", \
    aggfunc="size", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


#매출 집계


In [96]:
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", \
                              values="item_price", aggfunc="sum", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201902,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
201903,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
201905,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
201906,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
201907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


### 4: 상품명 오류 수정

In [97]:
# 상품: A~Z까지 26개임
print(len(pd.unique(uriage_data["item_name"])))

99


In [98]:
#상품명 오류: 공백처리, 대소문자 
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data.sort_values(by=["item_name"], ascending=True)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [99]:
print(pd.unique(uriage_data["item_name"]))
print(len(pd.unique(uriage_data["item_name"])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


### 5: 금액 결측치 수정

In [100]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [101]:
uriage_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   purchase_date   2999 non-null   datetime64[ns]
 1   item_name       2999 non-null   object        
 2   item_price      2612 non-null   float64       
 3   customer_name   2999 non-null   object        
 4   purchase_month  2999 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 117.3+ KB


In [102]:
#특이사항: 집계 기간에 상품 단가의 변동이 없다는 전제조건이 있음. 
flg_is_null = uriage_data["item_price"].isnull()
for idx, trg in enumerate(list(uriage_data.loc[flg_is_null, "item_name"].unique())):
    print(idx, trg)

0 상품S
1 상품A
2 상품P
3 상품N
4 상품W
5 상품R
6 상품I
7 상품L
8 상품F
9 상품O
10 상품B
11 상품C
12 상품V
13 상품Q
14 상품U
15 상품K
16 상품T
17 상품X
18 상품E
19 상품M
20 상품G
21 상품J
22 상품D
23 상품H
24 상품Y


### 6: 고객이름 오류 수정

### 7 : 날짜오류를 수정하자

### 8: 고객이름을 키로 두개의 데이터를 결합(조인)

### 9: 정제 데이터 덤프

### 10: 데이터를 집계